1. [교차 검증](#교차-검증)
1. [랜덤 서치](#랜덤-서치)
1. [하이퍼파라미터 튜닝](#--하이퍼파라미터-튜닝)

# 교차 검증과 그리드 서치

## - 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 나누어 ***검증 세트*** 라고 불리는 세트를 생성해 사용한다.
 - 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다. 
 - 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다. 
 - 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다. 
 - 그리고 마지막에 테스트 세트에서 최종 점수를 평가한다.


In [1]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head(3)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0


In [6]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [8]:
print(X_train.shape, X_test.shape)

(5197, 3) (1300, 3)


이 과정 다음에 X_train과 y_train 으로 훈련 세트 sub_input, sub_target 과 검증세트 val_input,val_target을 만든다. 

In [9]:
sub_input, val_input, sub_target, val_target = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [10]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


sub 세트를 이용해 이제 모델을 만들고 평가해 보자.

In [11]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(sub_input, sub_target)

print('훈련세트: ', dt_clf.score(sub_input, sub_target))
print('검증세트: ', dt_clf.score(val_input, val_target))

훈련세트:  0.9971133028626413
검증세트:  0.864423076923077


이 모델은 훈련 세트에 과대적합되어 있다. 매개변수를 바꿔서 더 좋은 모델을 찾아야 한다. 

## - 교차 검증

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용한다. 이렇게 하면 데이터의 80 ~ 90% 까지 훈련에 사용할 수 있다. 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

사이킷런에는 cross_validate() 라는 교차 검증 함수이용.

In [13]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt_clf, X_train, y_train)
print(scores)

{'fit_time': array([0.0046556 , 0.00456119, 0.00476003, 0.00501657, 0.00503016]), 'score_time': array([0.00039315, 0.00035453, 0.00045609, 0.000633  , 0.0006094 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키는 5개의 폴드 숫자가 담겨있다. cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행한다.

교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

In [14]:
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다. 

#### cv 매개변수

한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 
 - 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야 한다.

cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [17]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt_clf, X_train, y_train, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


훈련 세트를 섞은 후 10-폴드 교차 검증을 수행

In [19]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt_clf, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## - 하이퍼파라미터 튜닝

하이퍼파라미터는 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터이다.

1. 먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다. 
2. 그 다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔본다. 
   - 모델마다 적게는 1 ~ 2개에서, 만게는 5 ~ 6개의 매개변수를 제공한다.


#### GridSearchCV

사이킷런의 GridSearchCV 클래스는 친절하게도 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 별도로 cross_validate() 함수를 호출할 필요가 없다. 

기존 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아보자.

In [20]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도해보자. 

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

GridSearchCV의 cv 매개변수 기본값은 5이다. min_impurity_decrease 값 마다 5-폴드 교차 검증을 수행한다. 결국 5x5=25 개의 모델을 훈련하게 된다. 

 - n_jobs:CPU 코어 수. 기본값 1. -1 모든 코어 사용.

25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합 모델은 gs 객체의 bestestimator 속성에 저장 되어 있다.

In [22]:
dt = gs.best_estimator_
print(dt.score(X_train, y_train))

0.9615162593804117


In [23]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}



각 매개변수에서 수행한 교차 검증의 평균 점수는 cvresults 속성의 'mean_test_score' 키에 저장되어 있다. 

In [24]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


넘파이의 argmax() 함수를 사용 높은 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.

In [25]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


#### 규제 추가

조금 더 복잡한 매개 변수 조합을 탐색해보자. 
 - min_impurity_decreas는 노드를 분할하기 위한 불순도 감소 최소량을 지정
 - max_depth로 트리의 깊이를 제한
 - min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라보자.

In [27]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

이 매개변수로 수행할 교차 검증 횟수는 9x15x10 = 1350개이다. 기본 5-폴드 교차검증을 수행하므로 만들어지는 6750개이다. 

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최상의 매개변수 조합을 확인해 보자.

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### - 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴 때 **랜덤 서치** 를 사용하면 좋다.
 - 랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

In [32]:
from scipy.stats import uniform, randint

uniform과 randint 클래스는 모두 주어진 범위에서 고르게 정수, 실수 값을 뽑는다. 이를 균등 분포에서 샘플링한다 라고 한다.

In [33]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 7, 3, 7, 4, 5, 6, 3, 0, 7])

이번엔 1000개를 샘플링해서 각 숫자의 개수를 세어보자.

In [34]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96, 114, 101,  86,  96, 104, 105, 105,  95,  98]))

uniform 클래스의 사용법도 동일하다. 0 ~ 1 사이에서 10개의 실수를 추출해 보자.

In [35]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.01630635, 0.06857968, 0.56348102, 0.77859517, 0.11298253,
       0.07546262, 0.4688993 , 0.82480607, 0.41798123, 0.99894591])

#### RandomizedSearchCV 

 - min_samples_leaf : 리프 노드가 되기 위한 최소 샘플의 개수이다. 

In [36]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [37]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b556218b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b55621c10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b55621130>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b556083d0>},
                   random_state=42)

위 params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.

In [38]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 훈련 세트로 훈련되어 best_estimator_속성에 저장되어 있다.

In [40]:
dt = gs.best_estimator_

print(dt.score(X_test, y_test))

0.86


#### splitter 매개변수를 사용해 보자.

In [45]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b556218b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b55621c10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b55621130>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0b556083d0>},
                   random_state=42)

In [46]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(X_test, y_test))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


---

참고
 - 혼자 공부하는 머신러닝+딥러닝
